# Laboratorio 5 - Parte 2

### Máquinas de Vectores de Soporte

### 2018-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co

## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante:
Deiry Sofia Navas Muriel
#### Segundo Integrante:
David Alejandro Marin Alejandro

In [1]:
from __future__ import division
import numpy as np
import math
import matplotlib.pyplot as plt
#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")
%matplotlib inline

## Ejercicio 1

En este ejercicio usaremos la regresión por vectores de soporte para resolver el problema de regresión de la base de datos AirQuality (https://archive.ics.uci.edu/ml/datasets/Air+Quality).

La siguiente celda de código limpia la base de datos de todos sus datos faltantes y la deja lista en la variable DataBase.

In [2]:
#cargamos la bd de entrenamiento
db = np.loadtxt('BDatos/AirQuality.data',delimiter='\t')  # Assuming tab-delimiter
print ("Dim de la base de datos original: " + str(np.shape(db)))
db = db.reshape(9357,13)
DataBase = db
#La base d edatos AirQuality tiene muchos valores perdidos, vamos a quitar todos esos registros de la base de datos
#Están marcados como -200. Donde haya un valor -200 eliminaremos el registro.
j = 0
for i in range(0,np.size(db,0)):
    if -200 == db[i,12]:
        #print i
        j+=1
        DataBase = np.delete(DataBase,i,0)
    
print ("\nHay " + str(j) + " valores perdidos en la variable de salida. :(")

print ("\nDim de la base de datos sin las muestras con variable de salida perdido "+ str(np.shape(DataBase)))

#Ya hemos eliminado los registros con valor de la variable de salida perdido.

#Ahora vamos a imputar los valores perdidos en cada una de las características
print ("\nProcesando imputación de valores perdidos en las características . . .\n")

#data = open("AirQuality_cleaned.data", "w")

for k in range(0,np.size(DataBase,0)):
    for w in range(0,13):
        if -200 == DataBase[k,w]:
            DataBase[k,w] = round(np.mean(DataBase[:,w]))
        
    #data.write(str(DataBase[k,:]))

#data.close()
print ("Imputación finalizada.\n")

hay_missed_values = False
for i in range(0,np.size(DataBase,0)):
    if -200 in DataBase[i,:]:
        hay_missed_values = True
if(hay_missed_values):
    print ("Hay valores perdidos")
else:
    print ("No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase")

Dim de la base de datos original: (9357, 13)

Hay 366 valores perdidos en la variable de salida. :(

Dim de la base de datos sin las muestras con variable de salida perdido (8991, 13)

Procesando imputación de valores perdidos en las características . . .

Imputación finalizada.

No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase


In [3]:
X = DataBase[:,0:12]

Y = DataBase[:,12]

Definimos la función

In [4]:
#Mean Absolute Percentage Error para los problemas de regresión
def MAPE(Y_est,Y):
    ind = Y != 0 #Remueve los elementos que son cero en la variable deseada
    N = np.size(Y[ind])
    mape = np.sum(abs((Y_est[ind].reshape(N,1) - Y[ind].reshape(N,1))/(Y[ind].reshape(N,1)+np.finfo(np.float).eps)))/N
    return mape 

Complete el código siguiente para usar la regresión por vectores de suporte usando la librería sklearn. https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html

In [19]:
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.svm import SVR
import time


#Complete el código para crear el modelo SVM para regresión. 
#Use un kernel rbf con una malla de valores así: C en {0.1, 100} y gamma en {0.0001, 0.1}
#clf = ...
def main(kernel='rbf', C=100, gamma=1):
    tiempo_i = time.time()
    #Implemetamos la metodología de validación cross validation con 10 folds
    Folds = 5
    Errores = np.ones(Folds)
    j = 0
    kf = KFold(n_splits=Folds)
    percentVectors = np.zeros(Folds)

    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        #normalizamos los datos
        scaler = preprocessing.StandardScaler().fit(X_train)
        Xtrain = scaler.transform(X_train)
        Xtest = scaler.transform(X_test)

        #Complete el código
        clf = SVR(kernel=kernel, C=C, gamma=gamma, cache_size=200)
        
        # Entrenar el modelo
        #modelo = ...
        clf.fit(Xtrain,y_train)

        #Calculamos el porcentaje de vector de soporte
        percentVectors[j] = (len(clf.support_vectors_)/len(Xtrain))

        # Validación del modelo
        ypred = clf.predict(Xtest)

        Errores[j] = MAPE(ypred, y_test)
        j+=1

    # print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))
    # print ("Porcentaje de vectores de soporte",round(percentVectors.sum()/Folds, 5))
    # print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")
    
    mean = round(np.mean(Errores),4)
    std = round(np.std(Errores),4)
    percent = round(percentVectors.sum()/Folds, 4)
    return mean,std,percent

main()


Error de validación: 0.2259390395748797 +/- 0.11094914643434282
Porcentaje de vectores de soporte 0.21911


Tiempo total de ejecución: 6.830924987792969 segundos.


(0.2259, 0.1109, 0.2191)

In [22]:
kernels = ['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']
cs = [0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]
gammas = [0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1]
medias = np.zeros(len(kernels))
stds = np.zeros(len(kernels))
percents = np.zeros(len(kernels))
for i in range(0, len(kernels)):
    if kernels[i] is 'lineal':
        kernel = 'linear'
    elif kernels[i] is 'rbf':
        kernel =  'rbf'
    if gammas[i] is 0:
        gamma = 'auto'
    else:
        gamma = gammas[i]
    medias[i],stds[i],percents[i] = main(kernel=kernel, C=cs[i], gamma=gamma)
    print (str(medias[i]),str(stds[i]),str(percents[i]))


Error de validación: 0.21532161504323985 +/- 0.06460100562932213
Porcentaje de vectores de soporte 0.37863


Tiempo total de ejecución: 613.8920531272888 segundos.
0.2153 0.0646 0.3786

Error de validación: 0.24490897173183038 +/- 0.0962859267563447
Porcentaje de vectores de soporte 0.37666


Tiempo total de ejecución: 617.5645899772644 segundos.
0.2449 0.0963 0.3767

Error de validación: 0.27680077418621385 +/- 0.13461740462666608
Porcentaje de vectores de soporte 0.38497


Tiempo total de ejecución: 625.8208391666412 segundos.
0.2768 0.1346 0.385

Error de validación: 0.2869582909645992 +/- 0.15071982828475336
Porcentaje de vectores de soporte 0.38622


Tiempo total de ejecución: 664.3198959827423 segundos.
0.287 0.1507 0.3862


KeyboardInterrupt: 

## Ejercicio 2

Una vez complete el código, realice las simulaciones necesarias para llenar la tabla siguiente:

In [8]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(kernels),
    'C' : pd.Series(cs),
    'gamma' : pd.Series(gammas)})
df_types["MAPE Promedio"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)
df_types["MAPE Promedio"][23] = "0.2259"
df_types["Intervalo de confianza"][23] = "0.1109"
df_types["% de Vectores de Soporte"][23] = "0.2191"
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

/home/deiry/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/deiry/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/deiry/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/deiry/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/deiry/py3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/deiry/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

/home/deiry/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/deiry/py3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/deiry/py3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/deiry/py3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/deiry/py3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/deiry/py3/lib/python3.6/importlib/_bootstrap.py:21

ModuleNotFoundError: No module named 'qgrid'

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [59]:
qgrid_widget.get_changed_df()

MAPE Promedio Intervalo de confianza  \
Kernel C       gamma                                        
lineal 0.001   0.00                                         
       0.010   0.00                                         
       0.100   0.00                                         
       1.000   0.00                                         
       10.000  0.00                                         
       100.000 0.00                                         
rbf    0.001   0.01                                         
               0.10                                         
               1.00                                         
       0.010   0.01                                         
               0.10                                         
               1.00                                         
       0.100   0.01                                         
               0.10                                         
               1.00                                         
       1.000   0.01                                         
               0.10                                         
               1.00                                         
       10.000  0.01                                         
               0.10                                         
               1.00                                         
       100.000 0.01                                         
               0.10                                         
               1.00         0.2259                 0.1109   

                     % de Vectores de Soporte  
Kernel C       gamma                           
lineal 0.001   0.00                            
       0.010   0.00                            
       0.100   0.00                            
       1.000   0.00                            
       10.000  0.00                            
       100.000 0.00                            
rbf    0.001   0.01                            
               0.10                            
               1.00                            
       0.010   0.01                            
               0.10                            
               1.00                            
       0.100   0.01                            
               0.10                            
               1.00                            
       1.000   0.01                            
               0.10                            
               1.00                            
       10.000  0.01                            
               0.10                            
               1.00                            
       100.000 0.01                            
               0.10                            
               1.00                    0.2191

3.1 ¿Cuál es la finalidad de usar las funciones kernel en el modelo SVM?

R/:

3.2 ¿En este caso el porcentaje de vectores de soporte provee una información similar que en el problema de clasificación? Explique su respuesta.

R/:


3.3 Realice una gráfica de las salidas reales vs las predicciones del modelo SVM, para evaluar visualmente el desempeño del mismo. Esto solo para la configuración en la cuál se encontró el menor error.

In [7]:
'''Complete el código para hacer la gráfica aquí'''

'Complete el c\xc3\xb3digo para hacer la gr\xc3\xa1fica aqu\xc3\xad'